# Read and merge all CSV files from `data` (only import pandas)

This notebook reads every CSV in `Term-paper/data` using only `pandas` as an explicit import,
lists files using the Jupyter/IPython shell, then merges them into a single DataFrame.
If DataFrames share column names the merge uses those columns; otherwise DataFrames are concatenated side-by-side.

In [109]:
import pandas as pd
import os
import numpy as np

In [110]:
#1.1

data_path = 'data'

#  Step 1: Collect CSV files 
csv_files = [f for f in os.listdir(data_path) if f.lower().endswith('.csv')]
print(f"Found {len(csv_files)} CSV files")

dfs = []

#  Step 2: Load only datasets that have userid 
for fname in csv_files:
    full_path = os.path.join(data_path, fname)
    try:
        df = pd.read_csv(full_path, sep=';')
        if 'userid' in df.columns:
            df['source_file'] = fname  # tag for later identification
            dfs.append(df)
            print(f"Loaded {fname} ({df.shape}), has userid")
        else:
            print(f"Skipped {fname} (no userid column)")
    except Exception as e:
        print(f"Failed to read {fname}: {e}")

#  Step 3: Concatenate vertically (merging vertically) 
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    print(f"Final concatenated shape: {merged_df.shape}")


#  Step 4: Drop any unwanted column safely 
for col in merged_df.columns:
    if 'DATE,CPI' in col:
        merged_df = merged_df.drop(columns=[col])
        print(f"Dropped column: {col}")


Found 140 CSV files
Skipped CPI.csv (no userid column)
Loaded SCE-Apr-2014.csv ((1311, 30)), has userid
Loaded SCE-Apr-2015.csv ((1283, 30)), has userid
Loaded SCE-Apr-2016.csv ((1214, 30)), has userid
Loaded SCE-Apr-2017.csv ((1276, 30)), has userid
Loaded SCE-Apr-2018.csv ((1300, 30)), has userid
Loaded SCE-Apr-2019.csv ((1336, 30)), has userid
Loaded SCE-Apr-2020.csv ((1300, 30)), has userid
Loaded SCE-Apr-2021.csv ((1243, 30)), has userid
Loaded SCE-Apr-2022.csv ((1269, 30)), has userid
Loaded SCE-Apr-2023.csv ((1255, 30)), has userid
Loaded SCE-Apr-2024.csv ((1082, 30)), has userid
Loaded SCE-Aug-2013.csv ((1769, 30)), has userid
Loaded SCE-Aug-2014.csv ((1352, 30)), has userid
Loaded SCE-Aug-2015.csv ((1226, 30)), has userid
Loaded SCE-Aug-2016.csv ((1271, 30)), has userid
Loaded SCE-Aug-2017.csv ((1344, 30)), has userid
Loaded SCE-Aug-2018.csv ((1331, 30)), has userid
Loaded SCE-Aug-2019.csv ((1290, 30)), has userid
Loaded SCE-Aug-2020.csv ((1193, 30)), has userid
Loaded SCE-Aug

In [111]:
#1.2.1
print("Number of unique user IDs:", merged_df['userid'].nunique())

Number of unique user IDs: 23369


In [112]:
#1.2.2
print("Total rows:", len(merged_df))

Total rows: 176101


In [113]:
#1.2.3
print("Number of unique survey waves:", merged_df['source_file'].nunique())


Number of unique survey waves: 139


In [114]:
#1.2.4

# Extract month and year separately
merged_df[['month_str', 'year_str']] = merged_df['source_file'].str.extract(r'SCE-([A-Za-z]+)-(\d{4})')

# Combine and convert to datetime
merged_df['survey_date'] = pd.to_datetime(
    merged_df['month_str'] + ' ' + merged_df['year_str'],
    format='%b %Y',
    errors='coerce'  # ignore weird filenames
)

# Sanity check
print("Earliest date:", merged_df['survey_date'].min())
print("Latest date:", merged_df['survey_date'].max())

Earliest date: 2013-06-01 00:00:00
Latest date: 2024-12-01 00:00:00


In [115]:
#2.1

# Identify numeracy columns
num_lit_cols = [
    col for col in merged_df.columns
    if "num_lit_q" in col and "correct" in col
]
print("Numeracy columns:", num_lit_cols)

# Clean non-standard missing markers
merged_df[num_lit_cols] = merged_df[num_lit_cols].replace(
    ['', 'NA', 'N/A', 'null', 'None', 'nan', -99, 999],
    np.nan
)

# Sort so the first survey per user is first
merged_df = merged_df.sort_values(['userid', 'survey_date'])

# Extract each user’s first valid numeracy responses
first_vals = (
    merged_df.groupby('userid', sort=False)[num_lit_cols]
    .first()
    .add_suffix('_first')
    .reset_index()
)

# Merge those first-wave values back
merged_df = merged_df.merge(first_vals, on='userid', how='left')

# Fill missing numeracy values from the user’s first record
for col in num_lit_cols:
    merged_df[col] = merged_df[col].fillna(merged_df[f"{col}_first"])
    merged_df.drop(columns=f"{col}_first", inplace=True)

# Verification (fully vectorized)
rows_missing_after = merged_df[num_lit_cols].isnull().any(axis=1).sum()

# Users with all numeracy missing
users_all_missing_mask = merged_df[num_lit_cols].isnull().all(axis=1)
users_all_missing = merged_df.loc[users_all_missing_mask, 'userid'].nunique()

print(f"Rows with missing numeracy values after fill: {rows_missing_after}")
print(f"Users with all numeracy missing after fill: {users_all_missing}")


Numeracy columns: ['num_lit_q1_correct', 'num_lit_q2_correct', 'num_lit_q3_correct', 'num_lit_q5_correct', 'num_lit_q6_correct', 'num_lit_q8_correct', 'num_lit_q9_correct']
Rows with missing numeracy values after fill: 37378
Users with all numeracy missing after fill: 65


In [116]:
#2.2

# Step 1: Drop rows with missing demographics 
demographic_cols = ['female', 'age', 'educ']
merged_demo = merged_df.dropna(subset=demographic_cols)
after_demo = len(merged_demo)
print(f"After dropping missing {demographic_cols}: {after_demo} (Dropped {before_drop - after_demo})")

# Step 2: Drop rows with missing expectations 
expectation_cols = ['inflation', 'house_price_change', 'prob_stocks_up']
merged_expect = merged_demo.dropna(subset=expectation_cols)
after_expect = len(merged_expect)
print(f"After dropping missing {expectation_cols}: {after_expect} (Dropped {after_demo - after_expect})")

# Step 3: Drop rows with missing numeracy questions 
merged_final = merged_expect.dropna(subset=num_lit_cols)
after_num = len(merged_final)
print(f"After dropping missing numeracy columns: {after_num} (Dropped {after_expect - after_num})")



After dropping missing ['female', 'age', 'educ']: 175233 (Dropped 868)
After dropping missing ['inflation', 'house_price_change', 'prob_stocks_up']: 173550 (Dropped 1683)
After dropping missing numeracy columns: 137576 (Dropped 35974)


In [ ]:

#2.3.1

expectation_cols = ['inflation', 'house_price_change', 'prob_stocks_up']
expectation_cols = [c for c in expectation_cols if c in merged_df.columns]

total_below = 0

for col in expectation_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    q_low = merged_df[col].quantile(0.001)
    min_val = merged_df[col].min()

    n_below = (merged_df[col] < q_low).sum()
    merged_df = merged_df[merged_df[col] >= q_low]

    total_below += n_below
    print(f"{col}: Dropped {n_below} below {q_low:.2f} (min = {min_val})")

print(f"\nTotal observations dropped below 0.1th percentile: {total_below}")
print(f"Remaining dataset shape: {merged_df.shape}")


inflation: Dropped 139 below -50.00 (min = -60.0)
house_price_change: Dropped 162 below -35.00 (min = -45.0)
prob_stocks_up: Dropped 0 below 0.00 (min = 0.0)

Total observations dropped below 0.1th percentile: 301
Remaining dataset shape: (173446, 33)


In [121]:
#2.3.2
total_above = 0


for col in expectation_cols:
    q_high = merged_df[col].quantile(0.999)
    max_val = merged_df[col].max()

    n_above = (merged_df[col] > q_high).sum()
    merged_df = merged_df[merged_df[col] <= q_high]

    total_above += n_above
    print(f"{col}: Dropped {n_above} above {q_high:.2f} (max = {max_val})")

print(f"\nTotal observations dropped above 99.9th percentile: {total_above}")
print(f"Remaining dataset shape: {merged_df.shape}")


inflation: Dropped 0 above 100.00 (max = 100.0)
house_price_change: Dropped 0 above 100.00 (max = 100.0)
prob_stocks_up: Dropped 0 above 100.00 (max = 100.0)

Total observations dropped above 99.9th percentile: 0
Remaining dataset shape: (173446, 33)
